In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import cv2
from skimage.filters import frangi
from scipy.ndimage import minimum_filter
import pandas as pd


In [ ]:
def show(*args):
    images = len(args)
    rows = int(np.ceil(images / 5.0))   
    cols = min(5, images)              

    fig, ax = plt.subplots(rows, cols, figsize=(8 * cols, 8 * rows))

    for i, img in enumerate(args):
        if rows == 1:  
            if cols == 1: 
                ax.imshow(img, cmap='gray')
            else:  
                ax[i].imshow(img, cmap='gray')
        else: 
            row = i // 5
            col = i % 5
            ax[row, col].imshow(img, cmap='gray')

    if images < rows * cols:
        for i in range(images, rows * cols):
            fig.delaxes(ax.flatten()[i])
    plt.tight_layout()
    plt.show()

In [ ]:
def enhance_contrast(layer):
    if layer.dtype != np.uint8:
        layer = cv2.normalize(layer, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    equalized = cv2.equalizeHist(layer)
    return equalized

def emphasize_boundaries(layer):
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3, 3))
    eroded = cv2.erode(layer, kernel, iterations=1)
    dilated = cv2.dilate(eroded, kernel, iterations=1)
    return dilated

def combine_layers(layers):
    combined = np.maximum.reduce(layers)
    combined = cv2.normalize(combined, None, 0, 255, cv2.NORM_MINMAX, cv2.CV_8U)
    return combined

def process_image(image_path):
    image = cv2.imread(image_path, cv2.IMREAD_UNCHANGED)
    stacked_layers = cv2.split(image)
    tiff_stack = cv2.imreadmulti(image_path)[1]
    enhanced_layers = []
    for layer in stacked_layers:
        enhanced = enhance_contrast(layer)
        emphasized = emphasize_boundaries(enhanced)
        enhanced_layers.append(emphasized)

    bounded_img = tiff_stack[-1] - tiff_stack[1]
    bounded_img = cv2.GaussianBlur(bounded_img, (35,35), 10)
    bounded_img[bounded_img > 0] = 255

    final_output = combine_layers(enhanced_layers)        
    final_output[bounded_img == 0] = 0
    return final_output

def gamma_correction(img, gamma):
    return (np.power(img, gamma))*255

In [ ]:
def equalize_hist(img):
    hist, bins = np.histogram(img.flatten(), bins=256, range=[0,255])
    cdf = hist.cumsum()
    cdf_normalized = cdf * hist.max() / cdf.max()
    cdf_m = np.ma.masked_equal(cdf, 0)
    cdf_m = (cdf_m - cdf_m.min()) * 255 / (cdf_m.max() - cdf_m.min())
    cdf = np.ma.filled(cdf_m, 0).astype('uint8')
    
    img_out = cdf[img.flatten().astype('uint8')]
    img_out = np.reshape(img_out, img.shape)

    return img_out

def normalize(img):
    return (img - np.min(img))/(np.max(img)- np.min(img))*255

def convert_gray_to_rgb(image):
    image_rgb = np.stack((image,)*3, axis=-1)
    return image_rgb

def threshold_image(image, threshold=10, max_value=255):
    thresholded_image = np.where(image > threshold, max_value, 0).astype('uint8')
    return thresholded_image

In [ ]:
def create_cell_mask(thresh):
    num_labels, labels, stats, centroids = cv2.connectedComponentsWithStats(thresh, connectivity=8)

    min_fiber_length = 30
    cells_mask = np.zeros_like(thresh)

    for i in range(1, num_labels):
        if max(stats[i, cv2.CC_STAT_WIDTH], stats[i, cv2.CC_STAT_HEIGHT]) > min_fiber_length:
            cells_mask[labels == i] = 255
    return cells_mask

## Load image 1/4

In [ ]:
def image1_func():
    # Load image and normalize it
    image1 = process_image("pic1.TIF") / 255
    gamma = 3
    correct_image = gamma_correction(image1, gamma)
    image1_fibers = cv2.normalize(gamma_correction(image1, 2), None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
    normalized_image = cv2.normalize(correct_image, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)

    # Detect Cell borders
    blur = cv2.GaussianBlur(normalized_image, (7,)*2, 0)
    ret, thresh = cv2.threshold(blur, 80, 255, cv2.THRESH_BINARY_INV)

    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,)*2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel_open)
    
    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7,)*2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel_close)

    kernel_open = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5,)*2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel_open)

    kernel_close = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (9,)*2)
    thresh = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel_close)
    
    cells_mask = create_cell_mask(thresh)
    
    shape = cv2.MORPH_ELLIPSE
    kernel_size = (3,)*2
    kernel = cv2.getStructuringElement(shape, kernel_size)
    cells_mask = cv2.erode(cells_mask, kernel, iterations=2)

    kernel_size = (10,)*2
    kernel = cv2.getStructuringElement(shape, kernel_size)
    cells_mask = cv2.dilate(cells_mask, kernel)



### Detect cell borders

In [ ]:
blur = cv2.GaussianBlur(normalized_image, (7,)*2, 0)
ret, thresh = cv2.threshold(blur, 80, 255, cv2.THRESH_BINARY_INV)

show(normalized_image, blur, thresh)

In [ ]:

show(thresh, normalized_image)

In [ ]:

show(thresh)

In [ ]:

show(thresh)

In [ ]:

show(thresh)

In [ ]:


show(cells_mask)

In [ ]:

show(cells_mask)

In [ ]:
test = image1_fibers.copy()
test[cells_mask == 0] = 0
test[image1_fibers <= 30]=0
show(test, image1)

In [ ]:
equalized_image = equalize_hist(test)
filtered_frangi = frangi(equalized_image, np.arange(0.6,3, 0.3), black_ridges=False)
filtered_frangi1 = frangi(equalized_image, np.arange(0.5,3, 1.5), beta=0.3, black_ridges=False)
filtered_frangi2 = frangi(equalized_image, np.arange(1,5, 0.8), beta=0.3, gamma=0.7, black_ridges=False)

show(filtered_frangi, filtered_frangi1, filtered_frangi2)

In [ ]:
filtered_frangi[cells_mask == 0] = 0  
filtered_frangi1[cells_mask == 0] = 0 
filtered_frangi2[cells_mask == 0] = 0 

show(filtered_frangi, filtered_frangi1, filtered_frangi2)

In [ ]:
ed_frangi = cv2.dilate(filtered_frangi, (10,)*2)
ed_frangi = cv2.erode(ed_frangi, (5,)*2)

ed_frangi = cv2.dilate(ed_frangi, (9,)*2)
ed_frangi = cv2.erode(ed_frangi, (3,)*2)

ed_frangi = cv2.erode(ed_frangi, (5,)*2)
ed_frangi = cv2.dilate(ed_frangi, (5,)*2)

ed_frangi = cv2.erode(ed_frangi, (15,)*2)

show(ed_frangi)

In [ ]:
normalized_image_frangi = cv2.normalize(ed_frangi, None, alpha=0, beta=255, norm_type=cv2.NORM_MINMAX, dtype=cv2.CV_8U)
image = normalized_image_frangi
eq_image = cv2.equalizeHist(image)

hist,bins = np.histogram(image.flatten(),256,[0,256])
eq_hist,eq_bins = np.histogram(eq_image.flatten(),256,[0,256])

show(image, eq_image)

In [ ]:
equalized_image2 = equalize_hist(normalized_image)

image_color = convert_gray_to_rgb(equalized_image)

color_mask = np.zeros_like(image_color)
color_mask[eq_image > eq_image.mean()] = [0, 0, 255]
highlighted_image1 = cv2.addWeighted(image_color, 1.0, color_mask, 0.5, 0)

image_color = convert_gray_to_rgb(equalized_image2)

color_mask = np.zeros_like(image_color)
color_mask[eq_image > eq_image.mean()] = [0, 0, 255]
highlighted_image2 = cv2.addWeighted(image_color, 1.0, color_mask, 0.5, 0)
show(highlighted_image1, highlighted_image2)